# Lab 2 Text Data

Cameron Matson

## Purpose

This dataset is a collection reviews from Pitchfork, a music review site.  It 18,393 reviews contains reviews from 1999 to the present day.  They look at a variety There is a lot of research being done related to the way that people interact with music, specifically how they describe it in words.  What I'd like to get out of this data is if there is any relation between way the music is described in the reviews and the music itself.

## Data

The data was stored in a database in several tables, so first we need to restructure it.

In [76]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('../../data/reviews.csv')
print('number of reviews:',len(reviews))
reviews.head()

number of reviews: 18393


,reviewid,content
0,22703,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,"Eight years, five albums, and two EPs in, the ..."
2,22659,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,Kleenex began with a crash. It transpired one ...
4,22725,It is impossible to consider a given release b...


In [77]:
meta = pd.read_csv('../../data/meta.csv')
meta.info()
meta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 13 columns):
reviewid          18393 non-null int64
title             18391 non-null object
artist            18391 non-null object
url               18393 non-null object
score             18393 non-null float64
best_new_music    18393 non-null int64
author            18393 non-null object
author_type       14487 non-null object
pub_date          18393 non-null object
pub_weekday       18393 non-null int64
pub_day           18393 non-null int64
pub_month         18393 non-null int64
pub_year          18393 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.8+ MB


,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


All we want from the meta data is the score and best_new_music feature, because we want to see if we can label the reviewers feelings about the album from the text itself.

In [78]:
meta.drop(['title','artist', 'url', 'author', 'author_type', 'pub_date', 'pub_weekday', 'pub_day', 'pub_month', 'pub_year'], axis=1, inplace=True)
meta.info()
meta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 3 columns):
reviewid          18393 non-null int64
score             18393 non-null float64
best_new_music    18393 non-null int64
dtypes: float64(1), int64(2)
memory usage: 431.2 KB


,reviewid,score,best_new_music
0,22703,9.3,0
1,22721,7.9,0
2,22659,7.3,0
3,22661,9.0,1
4,22725,8.1,0


Finally we want the genre to see if we can infer that from the review

In [79]:
genres = pd.read_csv('../../data/genres.csv')

# some have multiple genres, for now lets just choose the first one
genres.drop_duplicates(subset=['reviewid'],inplace=True)

genres.info()
genres.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18389 entries, 0 to 22679
Data columns (total 2 columns):
reviewid    18389 non-null int64
genre       16024 non-null object
dtypes: int64(1), object(1)
memory usage: 431.0+ KB


,reviewid,genre
0,22703,electronic
1,22721,metal
2,22659,rock
3,22661,rock
4,22725,electronic


It appears that some of the reviews didn't have genres listed.  We'll drop these.

In [80]:
genres.dropna(inplace=True)
genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16024 entries, 0 to 22676
Data columns (total 2 columns):
reviewid    16024 non-null int64
genre       16024 non-null object
dtypes: int64(1), object(1)
memory usage: 375.6+ KB


In [81]:
df = pd.merge(genres, reviews, on='reviewid')
df = pd.merge(df, meta, on='reviewid')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16030 entries, 0 to 16029
Data columns (total 5 columns):
reviewid          16030 non-null int64
genre             16030 non-null object
content           16021 non-null object
score             16030 non-null float64
best_new_music    16030 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 751.4+ KB


,reviewid,genre,content,score,best_new_music
0,22703,electronic,"“Trip-hop” eventually became a ’90s punchline,...",9.3,0
1,22721,metal,"Eight years, five albums, and two EPs in, the ...",7.9,0
2,22659,rock,Minneapolis’ Uranium Club seem to revel in bei...,7.3,0
3,22661,rock,Kleenex began with a crash. It transpired one ...,9.0,1
4,22725,electronic,It is impossible to consider a given release b...,8.1,0


Looks like there were still a few reviews that didn't match up in all three tables, or that there were some null values in which if fine, we'll just drop to the lowest common denominator

In [82]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16021 entries, 0 to 16029
Data columns (total 5 columns):
reviewid          16021 non-null int64
genre             16021 non-null object
content           16021 non-null object
score             16021 non-null float64
best_new_music    16021 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 751.0+ KB


That's a lot of reviews, for efficiency's sake lets just take a sample

In [83]:
df = df.sample(n=1000).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 6976 to 2070
Data columns (total 5 columns):
reviewid          1000 non-null int64
genre             1000 non-null object
content           1000 non-null object
score             1000 non-null float64
best_new_music    1000 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 46.9+ KB


### Bag of Words Model


In [84]:
from sklearn.feature_extraction.text import CountVectorizer

# make a CountVectorizer object
counter = CountVectorizer(stop_words='english')

all_text = df.content.tolist()

# use counter object to transorm the text in to BOW
bag_words = counter.fit_transform(all_text)
print(bag_words.shape)

(1000, 36564)


In [85]:
#print(counter.vocabulary_)

That's a lot of words.  How might we be able to reduce it?  The problem is that the most common way that people talk about music is through metaphor, so it makes it difficult to choose a narrow vocabulary.  For the purpose of this analysis I'm going to do a few things though:
- Proper Nouns: most of these will probably correspond to artists and the titles of songs and albums
- Numbers: while it actually might be interesting to include dates, the majority of digits in this dataset are probably not very useful
- Non-English words: again would be nice ESPECIALLY when talking about the actual music itself (music lexicon has a lot non-english words), but since pitchfork is a english language site, and primarily concerned with popular music as opposed to classical art music, there probably won't be too many foreign words to begin with

Those three criteria should be pretty well contained in the Scrabble dictionary.

In [86]:
with open('../../data/scrabble.txt') as f:
    vocab = f.read().split('\n')
    
counter = CountVectorizer(stop_words='english', vocabulary=vocab) # an object capable of counting words in a document!
bag_words = counter.fit_transform(all_text)
print(bag_words.shape)

(1000, 79340)


But wait!  That looks like we just increased the number of words in the dataset.  Well we did, but that's because now its checking for almost every word in the English language, we can get rid of columns (words) that don't show up in the data.

In [87]:
df_bow = pd.DataFrame(data=bag_words.toarray(), columns=counter.get_feature_names())

# this will return only columns that aren't all 0
df_bow = df_bow.loc[:, (df_bow != 0).any(axis=0)]
df_bow.shape

(1000, 17558)

Look at that, it got smaller.

In [88]:
df_bow

,abacus,abandon,abandons,abattoir,abba,abbey,abbeys,abdomen,aberrant,abetted,...,zoned,zones,zoning,zoo,zoom,zooming,zooms,zydeco,zygotes,zygotic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Better.

### TF-IDF
A more useful representation of the data would be TFIDF

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfer = TfidfVectorizer(stop_words='english', vocabulary=vocab)


tfidf = tfidfer.fit_transform(all_text)
print(tfidf.shape)

(1000, 79340)


In [118]:
df_tfidf = pd.DataFrame(data=tfidf.toarray(),columns=tfidfer.get_feature_names())
df_tfidf = df_tfidf.loc[:, (df_tfidf != 0).any(axis=0)]

df_tfidf.head()


,abacus,abandon,abandons,abattoir,abba,abbey,abbeys,abdomen,aberrant,abetted,...,zoned,zones,zoning,zoo,zoom,zooming,zooms,zydeco,zygotes,zygotic
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.140724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.088842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
df_tfidf.max().sort_values()[-10:]

visitor     0.771865
volta       0.781227
manning     0.793446
mason       0.802722
lungfish    0.804572
frank       0.812817
askew       0.820677
bey         0.824209
walker      0.834419
lust        0.846817
dtype: float64

In [124]:
test = df.iloc[0].copy()

In [125]:
test

reviewid                                                      13901
genre                                                          rock
content           Jimmy LaValle's career as the Album Leaf took ...
score                                                           6.3
best_new_music                                                    0
Name: 6976, dtype: object